In [1]:
from pyspark.sql import SparkSession
from pyspark.sql.types import StructType, StructField
from pyspark.sql.types import StringType, IntegerType, DateType, DoubleType
from pyspark.sql import functions

In [2]:
spark = SparkSession \
    .builder \
    .appName("airlinedelay") \
    .getOrCreate()

context = spark.sparkContext

22/12/18 14:03:59 WARN Utils: Your hostname, lucian resolves to a loopback address: 127.0.1.1; using 192.168.100.7 instead (on interface eno1)
22/12/18 14:03:59 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address


Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


22/12/18 14:04:00 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [3]:
schema = StructType([
    StructField("FL_DATE", DateType(), True),
    StructField("OP_CARRIER", StringType(), True),
    StructField("OP_CARRIER_FL_NUM", IntegerType(), True),
    StructField("ORIGIN", StringType(), True),
    StructField("DEST", StringType(), True),
    StructField("CRS_DEP_TIME", DoubleType(), True),
    StructField("DEP_TIME", DoubleType(), True),
    StructField("DEP_DELAY", DoubleType(), True),
    StructField("TAXI_OUT", DoubleType(), True),
    StructField("WHEELS_OFF", DoubleType(), True),
    StructField("WHEELS_ON", DoubleType(), True),
    StructField("TAXI_IN", DoubleType(), True),
    StructField("CRS_ARR_TIME", DoubleType(), True),
    StructField("ARR_TIME", DoubleType(), True),
    StructField("ARR_DELAY", DoubleType(), True),
    StructField("CANCELLED", DoubleType(), True),
    StructField("CANCELLATION_CODE", StringType(), True),
    StructField("DIVERTED", DoubleType(), True),
    StructField("CRS_ELAPSED_TIME", DoubleType(), True),
    StructField("ACTUAL_ELAPSED_TIME", DoubleType(), True),
    StructField("AIR_TIME", DoubleType(), True),
    StructField("DISTANCE", DoubleType(), True),
    StructField("CARRIER_DELAY", DoubleType(), True),
    StructField("WEATHER_DELAY", DoubleType(), True),
    StructField("NAS_DELAY", DoubleType(), True),
    StructField("SECURITY_DELAY", DoubleType(), True),
    StructField("LATE_AIRCRAFT_DELAY", DoubleType(), True)
])

df = spark.read.format("csv") \
    .option("sep", ",") \
    .option("header", True) \
    .schema(schema) \
    .load("*.csv")

df.createOrReplaceTempView("airlinedelay")

22/12/18 14:04:08 WARN package: Truncated the string representation of a plan since it was too large. This behavior can be adjusted by setting 'spark.sql.debug.maxToStringFields'.


In [34]:
df.show()

22/12/18 16:29:47 WARN CSVHeaderChecker: Number of column in CSV header is not equal to number of fields in the schema:
 Header length: 28, schema size: 27
CSV file: file:///home/lucian/Facultate/BDA/source/2012.csv
+----------+----------+-----------------+------+----+------------+--------+---------+--------+----------+---------+-------+------------+--------+---------+---------+-----------------+--------+----------------+-------------------+--------+--------+-------------+-------------+---------+--------------+-------------------+
|   FL_DATE|OP_CARRIER|OP_CARRIER_FL_NUM|ORIGIN|DEST|CRS_DEP_TIME|DEP_TIME|DEP_DELAY|TAXI_OUT|WHEELS_OFF|WHEELS_ON|TAXI_IN|CRS_ARR_TIME|ARR_TIME|ARR_DELAY|CANCELLED|CANCELLATION_CODE|DIVERTED|CRS_ELAPSED_TIME|ACTUAL_ELAPSED_TIME|AIR_TIME|DISTANCE|CARRIER_DELAY|WEATHER_DELAY|NAS_DELAY|SECURITY_DELAY|LATE_AIRCRAFT_DELAY|
+----------+----------+-----------------+------+----+------------+--------+---------+--------+----------+---------+-------+------------+------

GENERAL EVOLUTION OF DELAYS/IMMEDIACIES FOR DEPARTURES AND ARRIVALS

In [30]:
spark.sql("SELECT EXTRACT(year from FL_DATE) AS Year, AVG(IF(DEP_DELAY > -1, DEP_DELAY, NULL)) AS AverageDepartureDelayPerYear, AVG(IF(DEP_DELAY < 1, DEP_DELAY, NULL)) AS AverageDepartureImmediacyPerYear, AVG(IF(ARR_DELAY > -1, ARR_DELAY, NULL)) AS AverageArrivalDelayPerYear, AVG(IF(ARR_DELAY < 1, ARR_DELAY, NULL)) AS AverageArrivalImmediacyPerYear \
    FROM airlinedelay \
    GROUP BY Year") \
    .show()

+----+----------------------------+--------------------------------+--------------------------+------------------------------+
|Year|AverageDepartureDelayPerYear|AverageDepartureImmediacyPerYear|AverageArrivalDelayPerYear|AverageArrivalImmediacyPerYear|
+----+----------------------------+--------------------------------+--------------------------+------------------------------+
|2012|          25.064444508406826|              -4.319753328392286|        28.104134902282805|           -11.559131542244033|
|2013|          27.491661013175978|              -4.447244216159287|        30.017653221108908|           -11.230981537304695|
+----+----------------------------+--------------------------------+--------------------------+------------------------------+



AVERAGE DEPARTURE/ARRIVAL DELAYS AND IMMEDIACIES FOR EACH CARRIER

In [33]:
spark.sql("SELECT OP_CARRIER AS Carrier, AVG(IF(DEP_DELAY > -1, DEP_DELAY, NULL)) AS AverageDepartureDelayPerYear, AVG(IF(DEP_DELAY < 1, DEP_DELAY, NULL)) AS AverageDepartureImmediacyPerYear, AVG(IF(ARR_DELAY > -1, ARR_DELAY, NULL)) AS AverageArrivalDelayPerYear, AVG(IF(ARR_DELAY < 1, ARR_DELAY, NULL)) AS AverageArrivalImmediacyPerYear \
    FROM airlinedelay \
    GROUP BY OP_CARRIER \
    ORDER BY OP_CARRIER") \
    .show(1000)

+-------+----------------------------+--------------------------------+--------------------------+------------------------------+
|Carrier|AverageDepartureDelayPerYear|AverageDepartureImmediacyPerYear|AverageArrivalDelayPerYear|AverageArrivalImmediacyPerYear|
+-------+----------------------------+--------------------------------+--------------------------+------------------------------+
|     9E|           36.53694558651262|              -5.307216630484836|        33.862480182668435|           -13.048573532213304|
|     AA|          27.668853203873958|              -4.157916455312187|        30.842636470073337|           -12.191846663345155|
|     AS|            21.7290827904253|              -6.230512677614042|        20.253859198604584|           -12.419562507973582|
|     B6|           34.15038997330664|              -5.165645598285414|         35.70338399042472|           -12.823157498410488|
|     DL|          24.294510015871627|             -3.7388303717402374|        27.18404159

EVOLUTION OF DELAYS/IMMEDIACIES FOR DEPARTURES AND ARRIVALS PER CARRIER FOR EACH YEAR

In [31]:
spark.sql("SELECT OP_CARRIER AS Carrier, EXTRACT(year from FL_DATE) AS Year, AVG(IF(DEP_DELAY > -1, DEP_DELAY, NULL)) AS AverageDepartureDelayPerYear, AVG(IF(DEP_DELAY < 1, DEP_DELAY, NULL)) AS AverageDepartureImmediacyPerYear, AVG(IF(ARR_DELAY > -1, ARR_DELAY, NULL)) AS AverageArrivalDelayPerYear, AVG(IF(ARR_DELAY < 1, ARR_DELAY, NULL)) AS AverageArrivalImmediacyPerYear \
    FROM airlinedelay \
    GROUP BY OP_CARRIER, Year  \
    ORDER BY OP_CARRIER, Year") \
    .show(1000)

+-------+----+----------------------------+--------------------------------+--------------------------+------------------------------+
|Carrier|Year|AverageDepartureDelayPerYear|AverageDepartureImmediacyPerYear|AverageArrivalDelayPerYear|AverageArrivalImmediacyPerYear|
+-------+----+----------------------------+--------------------------------+--------------------------+------------------------------+
|     9E|2013|           36.53694558651262|              -5.307216630484836|        33.862480182668435|           -13.048573532213304|
|     AA|2012|          28.027254955743352|               -4.14197884082297|        30.108259241108932|           -12.072422818092937|
|     AA|2013|           27.35091324112733|              -4.174709671368422|        31.600118245454095|            -12.30414137999665|
|     AS|2012|           23.53503184713376|              -6.184424957662059|         22.52250832019618|           -12.619993043881514|
|     AS|2013|          20.017962364569474|            